In [17]:
%pylab inline
import warnings
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

In [26]:
# Importing the datasets
reviews = pd.read_csv('reviews_restaurants_text.csv')
reviews['text'] = reviews['text'].str[2:-2]

#Undersampling of the dataset to get a balanced dataset
review1 = reviews[reviews['stars'] == 1][0:12000]
review2 = reviews[reviews['stars'] == 2][0:7000]
review3 = reviews[reviews['stars'] == 3][0:12000]
review4 = reviews[reviews['stars'] == 4][0:12000]
review5 = reviews[reviews['stars'] == 5][0:12000]
frames = [review1, review2, review3,review4,review5]
reviews = pd.concat(frames)
len(reviews)
reviews.head()

,review_id,business_id,user_id,text,stars
1,xryg94pDLOO71veGcQINuQ,dfRAK2mgdHbL2_YsFqtCdQ,tL2pS5UOmN6aAOi3Z-qFGg,I used to come here back in the 90s when this ...,1
14,fx0AFrBZ1246jKGLKBgtMA,VPO8pBUwYz1u6GoG0d2U-Q,Tr1ulYYAHAw5FzEQQwZEJQ,I really like the chicken and biscuits. But it...,1
16,dH4h3Worla2GcuZxJvnlVQ,gsjxrwdHqKdTeNx3GeDKNQ,1OuFRzppeTtaEEWvc82J-A,I asked for extra sauce and got zero sauce sho...,1
21,25hX4UzYNqmsy_YdTdzvjg,JyxHvtj-syke7m9rbza7mA,RFIZhrka0gCpiL62M5E93g,I went to this place to see what all the hype ...,1
23,NJjjaMAZDnB3doJvX28SJg,QIn3PuGUkinSibUn-NsljQ,V3gzoLZb9zGwfSBqGDOTXg,I got the nachos with carne asada. There was n...,1


In [31]:
reviews['user_id'] = pd.factorize(reviews.user_id)[0]
reviews['business_id'] = pd.factorize(reviews.business_id)[0]

In [32]:
unique_users = reviews.user_id.unique().shape[0]
unique_restaurants = reviews.business_id.unique().shape[0]

In [33]:
from sklearn import cross_validation as cv
train_data, test_data = cv.train_test_split(reviews, test_size=0.20)

#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((unique_users, unique_restaurants))
# train_data_matrix
for line in train_data.itertuples():
#      print(line[5])
     train_data_matrix[line[3], line[2]] = line[5]

test_data_matrix = np.zeros((unique_users, unique_restaurants))
for line in test_data.itertuples():
    test_data_matrix[line[3], line[2]] = line[5]

In [ ]:
from sklearn.metrics.pairwise import pairwise_distances
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')
item_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

In [8]:
def predict(ratings, similarity, type='user'):
    if type == 'user':
        mean_user_rating = ratings.mean(axis=1)
        #You use np.newaxis so that mean_user_rating has same format as ratings
        ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
        pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T
    elif type == 'item':
        pred = ratings.dot(similarity) / np.array([np.abs(similarity).sum(axis=1)])
    return pred

item_prediction = predict(train_data_matrix, item_similarity, type='item')
user_prediction = predict(train_data_matrix, user_similarity, type='user')

In [9]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

print ('User-based CF RMSE: ' + str(rmse(user_prediction, test_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, test_data_matrix)))


print ('User-based CF RMSE: ' + str(rmse(user_prediction, train_data_matrix)))
print ('Item-based CF RMSE: ' + str(rmse(item_prediction, train_data_matrix)))

User-based CF RMSE: 3.382852990688319
Item-based CF RMSE: 3.384852488490991
User-based CF RMSE: 3.415862737871881
Item-based CF RMSE: 3.416699872099379
